<a href="https://colab.research.google.com/github/vaibhavrouduri/Thesis-Work/blob/main/Week1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import numpy as np
import random
import time
from itertools import filterfalse

Imported pretrained glove vectors trained on 800B words.
Took help from https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/nlp/ipynb/pretrained_word_embeddings.ipynb#scrollTo=EVOo9FVY2L8P and https://medium.com/analytics-vidhya/basics-of-using-pre-trained-glove-vectors-in-python-d38905f356db


---



In [2]:
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip
!unzip -q glove.840B.300d.zip

--2021-08-12 06:35:44--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2021-08-12 06:35:44--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2021-08-12 06:35:45--  http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip

In [3]:
embeddings_index = {}
with open("glove.840B.300d.txt") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  """


#Task 1

In [4]:
def cos(v1, v2):
  num = np.dot(v1, v2)
  denom = np.linalg.norm(v1) * np.linalg.norm(v2)
  return num/denom

In [5]:
def weat_swAB(w, A, B):  # w is a single word, A and B are sets of attribute words
  A_list = []
  B_list = []

  for v in A:
    A_list.append(cos(embeddings_index[w], embeddings_index[v]))
  for v in B:
    B_list.append(cos(embeddings_index[w], embeddings_index[v]))

  A_array = np.array(A_list)
  B_array = np.array(B_list)

  return np.mean(A_array) - np.mean(B_array)

In [6]:
def weat_ES(X, Y, A, B):
  x_list = []
  y_list = []

  for x in X:
    x_list.append(weat_swAB(x, A, B))
  for y in Y:
    y_list.append(weat_swAB(y, A, B))

  x_array = np.array(x_list)
  y_array = np.array(y_list)

  return (np.mean(x_array) - np.mean(y_array))/(np.std(np.concatenate((x_array, y_array))))

In [7]:
X = ["woman", "mother"]
Y = ["man", "father"]
A = ["health", "happy"]
B = ["pollute", "tragedy"]
X_Y = X + Y

In [ ]:

# X = ["bagpipe", "cello", "guitar", "lute", "trombone", "banjo", "clarinet", "harmonica", "mandolin", "trumpet", "bassoon", "drum", "harp", "oboe", "tuba", "bell", "fiddle", "harpsichord", "piano", "viola", "bongo",
# "flute", "horn", "saxophone", "violin"] #Instruments
# Y = ["arrow", "club", "gun", "missile", "spear", "axe", "dagger", "harpoon", "pistol", "sword", "blade", "dynamite", "hatchet", "rifle", "tank", "bomb", "firearm", "knife", "shotgun", "teargas", "cannon", "grenade",
#     "mace", "slingshot", "whip"] #Weapons
# A = ["caress", "freedom", "health", "love", "peace", "cheer", "friend", "heaven", "loyal", "pleasure", "diamond", "gentle", "honest", "lucky", "rainbow", "diploma", "gift", "honor", "miracle", "sunrise", "family",
#     "happy", "laughter", "paradise", "vacation"] #Pleasant
# B = ["abuse", "crash", "filth", "murder", "sickness", "accident", "death", "grief", "poison", "stink", "assault", "disaster", "hatred", "pollute", "tragedy", "divorce", "jail", "poverty", "ugly", "cancer", "kill", "rotten",
#     "vomit", "agony", "prison"]

In [8]:
weat_ES(X, Y, A, B)

1.3516492

In [9]:
def test_statistic(X, Y, A, B):
  x_list = []
  y_list = []

  for x in X:
    x_list.append(weat_swAB(x, A, B))
  for y in Y:
    y_list.append(weat_swAB(y, A, B))

  x_array = np.array(x_list)
  y_array = np.array(y_list)

  return np.sum(x_array) - np.sum(y_array)

In [10]:
test_statistic(X, Y, A, B)

0.075883955

In [11]:
def random_permutation(targets, r):
  return random.sample(targets, r)

Took help from https://github.com/jsedoc/ConceptorDebias/blob/master/WEAT/WEAT.ipynb

In [12]:
def p_value_weat(X, Y, A, B, iterations):
  X_Y = X + Y
  size_of_permutation = len(X)
  permutations = []
  test_statistics_permutations = []
  condition_satisfied = []

  while iterations != 0:
    permutations.append(random_permutation(X_Y, size_of_permutation))
    iterations = iterations - 1

  for Xi in permutations:
    Yi = filterfalse(lambda w:w in Xi, X_Y)
    Xi_list = []
    Yi_list = []
    for w in Xi:
      Xi_list.append(embeddings_index[w])
    for w in Yi:
      Yi_list.append(embeddings_index[w])
    Xi_array = np.array(Xi_list)
    Yi_array = np.array(Yi_list)
    test_statistics_permutations.append(test_statistic(Xi, Yi, A, B))

  overall_test_statistic = test_statistic(X, Y, A, B)

  condition_satisfied = [p > overall_test_statistic for p in test_statistics_permutations]

  condition_satisfied_array = np.array(condition_satisfied)

  return condition_satisfied_array.sum()/condition_satisfied_array.size

In [13]:
p_value_weat(X, Y, A, B, 1000) # Normal distribution 1000 iterations

1.0

In [14]:
p_value_weat(X, Y, A, B, 1000000) # Empirical distribution 1,000,000 iterations

1.0

#Task 2

In [15]:
def wefat_ES(w, A, B):
  A_B = A + B
  cos_A_B = []
  for x in A_B:
    cos_A_B.append(cos(embeddings_index[w], embeddings_index[x]))
  cos_A_B_array = np.array(cos_A_B)

  return weat_swAB(w, A, B)/np.std(cos_A_B_array)

In [ ]:
A1 = ["caress", "freedom", "health", "love", "peace", "cheer", "friend", "heaven", "loyal", "pleasure", "diamond", "gentle", "honest", "lucky", "rainbow", "diploma", "gift", "honor", "miracle", "sunrise", "family", "happy", "laughter", "paradise", "vacation"]
A2 = ["abuse", "crash", "filth", "murder", "sickness", "accident", "death", "grief", "poison", "stink", "assault", "disaster", "hatred", "pollute", "tragedy", "divorce", "jail", "poverty", "ugly", "cancer", "kill", "rotten", "vomit", "agony", "prison"]

In [16]:
# FA=["female", "woman", "girl", "sister", "she", "her", "hers", "daughter"]
# FB=["male", "man", "boy", "brother", "he", "him", "his", "son"]

In [18]:
def p_value_wefat(w, A, B, iterations):
  A_B = A + B
  size_of_permutation = len(A)
  permutations = []
  test_statistics_permutations = []
  condition_satisfied = []

  while iterations != 0:
    permutations.append(random_permutation(A_B, size_of_permutation))
    iterations = iterations - 1

  for Ai in permutations:
    Bi = filterfalse(lambda w:w in Ai, A_B)
    Ai_list = []
    Bi_list = []
    for w in Ai:
      Ai_list.append(embeddings_index[w]) 
    for w in Bi:
      Bi_list.append(embeddings_index[w])
    Ai_array = np.array(Ai_list)
    Bi_array = np.array(Bi_list)
    test_statistics_permutations.append(weat_swAB(w, Ai, Bi))

  overall_test_statistic = weat_swAB(w, A, B)

  condition_satisfied = [p > overall_test_statistic for p in test_statistics_permutations]

  condition_satisfied_array = np.array(condition_satisfied)

  return condition_satisfied_array.sum()/condition_satisfied_array.size

In [ ]:
wefat_ES("XXXXX", A1, A2)

0.17482777

In [ ]:
p_value_wefat("XXXXX", A1, A2, 1000000)

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0.0

In [ ]:
wefat_ES("weat", A1, A2)

-0.15603785

In [ ]:
p_value_wefat("weat", A1, A2, 1000000)

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0.0

In [ ]:
wefat_ES("AylinCaliskan", A1, A2)

In [ ]:
p_value_wefat("AylinCaliskan", A1, A2, 1000000)